In [6]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
from dateutil import parser

In [2]:
store_status = pd.read_csv('./store status.csv')
store_status.head()

,store_id,status,timestamp_utc
0,8419537941919820732,active,2023-01-22 12:09:39.388884 UTC
1,54515546588432327,active,2023-01-24 09:06:42.605777 UTC
2,8377465688456570187,active,2023-01-24 09:07:26.441407 UTC
3,5955337179846162144,active,2023-01-24 09:08:07.634899 UTC
4,1169347689335808384,active,2023-01-24 09:08:18.436854 UTC


In [8]:
store_status.shape

(1822080, 3)

In [10]:
store_status.iloc[:, 'timestamp_utc'] = store_status.timestamp_utc.apply(lambda x: parser.parse(x))
store_status.head()

C:\Users\tusha\AppData\Local\Temp\ipykernel_12144\247591809.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  store_status.loc[:, 'timestamp_utc'] = store_status.timestamp_utc.apply(lambda x: parser.parse(x))


,store_id,status,timestamp_utc
0,8419537941919820732,active,2023-01-22 12:09:39.388884+00:00
1,54515546588432327,active,2023-01-24 09:06:42.605777+00:00
2,8377465688456570187,active,2023-01-24 09:07:26.441407+00:00
3,5955337179846162144,active,2023-01-24 09:08:07.634899+00:00
4,1169347689335808384,active,2023-01-24 09:08:18.436854+00:00


In [36]:
for i,r in store_status.groupby('store_id'):
    if r[r['status']=='inactive'].shape[0]>1:
        print(r.sort_values(['timestamp_utc']).to_csv('./temp.csv'))
        break

None


In [21]:
menu_hours = pd.read_csv('./Menu hours.csv')
menu_hours.head()

,store_id,day,start_time_local,end_time_local
0,1481966498820158979,4,00:00:00,00:10:00
1,1481966498820158979,2,00:00:00,00:10:00
2,1481966498820158979,0,00:00:00,00:10:00
3,1481966498820158979,1,00:00:00,00:10:00
4,1481966498820158979,5,00:00:00,00:10:00


In [41]:
menu_hours.shape

(86198, 4)

In [22]:
menu_hours.start_time_local.fillna('00:00:00', inplace=True)
menu_hours.end_time_local.fillna('23:59:59', inplace=True)
menu_hours.head()

,store_id,day,start_time_local,end_time_local
0,1481966498820158979,4,00:00:00,00:10:00
1,1481966498820158979,2,00:00:00,00:10:00
2,1481966498820158979,0,00:00:00,00:10:00
3,1481966498820158979,1,00:00:00,00:10:00
4,1481966498820158979,5,00:00:00,00:10:00


In [32]:
for i,r in menu_hours.groupby(['store_id']):
    if r.shape[0]>1:
        print(r)
        break

              store_id  day start_time_local end_time_local
69775  257406274356679    6         00:00:00       23:59:00
69776  257406274356679    3         00:00:00       23:59:00
69777  257406274356679    2         00:00:00       23:59:00
69778  257406274356679    5         00:00:00       23:59:00
69779  257406274356679    0         00:00:00       23:59:00
69780  257406274356679    4         00:00:00       23:59:00
69781  257406274356679    1         00:00:00       23:59:00


C:\Users\tusha\AppData\Local\Temp\ipykernel_12144\3988413974.py:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i,r in menu_hours.groupby(['store_id']):


In [4]:
bq_results = pd.read_csv('./bq-results.csv')
bq_results.head()

,store_id,timezone_str
0,8139926242460185114,Asia/Beirut
1,5415949628544298339,America/Boise
2,3408529570017053440,America/Denver
3,9055649751952768824,America/Denver
4,4428372089193592098,America/Denver


In [11]:
bq_results.timezone_str.fillna('America/Chicago', inplace=True)
bq_results.head()

,store_id,timezone_str
0,8139926242460185114,Asia/Beirut
1,5415949628544298339,America/Boise
2,3408529570017053440,America/Denver
3,9055649751952768824,America/Denver
4,4428372089193592098,America/Denver


In [39]:
conn = sqlite3.connect('../db.sqlite3')

In [40]:
conn.execute('''
SELECT name FROM sqlite_master  
  WHERE type='table';
''').fetchall()

[('django_migrations',),
 ('sqlite_sequence',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('django_admin_log',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_user',),
 ('django_session',),
 ('main_store',),
 ('main_storestatus',),
 ('main_businesshours',)]

In [38]:
conn.close()

In [ ]:
for i,r in bq_results.iterrows():
    t = Store.objects.create(store_id = int(r['store_id']), timezone = r['timezone_str'])
    t.save()

In [ ]:
for i,r in menu_hours.iterrows():
    store = Store.objects.get_or_create(store_id = int(r['store_id']))[0]
    # https://stackoverflow.com/questions/31304890/wrong-aware-datetime-with-pytz-and-america-chicago
    import pytz
    BusinessHours.objects.get_or_create(
        store_id = store, 
        start_time = parser.parse(r['start_time_local']).replace(tzinfo=pytz.timezone(store.timezone)), 
        end_time = parser.parse(r['end_time_local']).replace(tzinfo=pytz.timezone(store.timezone)),
        day_of_week = r['day']
    )[0].save()

    # utc_date_time = date_time.astimezone(pytz.utc)


In [ ]:
for i,r in store_status.iterrows():
    store = Store.objects.get_or_create(store_id = int(r['store_id']))[0]
    # https://stackoverflow.com/questions/31304890/wrong-aware-datetime-with-pytz-and-america-chicago
    import pytz
    StoreStatus.objects.get_or_create(
        store_id = store, 
        timestamp = r['timestamp_utc'].replace(
            tzinfo=pytz.timezone(store.timezone)
            ).astimezone(pytz.utc),
        status = r['status']
    )[0].save()

    # utc_date_time = date_time.astimezone(pytz.utc)
